In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import streamlit as st

df = pd.read_csv("/Users/tyska/DataProjects/carsalesads/vehicles_us.csv")


In [2]:
#filling missing values 
df['model_year'] = df.groupby('model')['model_year'].transform(lambda x: x.fillna(x.median()))

df['cylinders'] = df.groupby('model')['cylinders'].transform(lambda x: x.fillna(x.median()))

df['odometer'] = df.groupby('model_year')['odometer'].transform(lambda x: x.fillna(x.median()))

df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    51525 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     51525 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      51524 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


/Users/tyska/DataProjects/venv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


(None,
    price  model_year           model  condition  cylinders fuel  odometer  \
 0   9400      2011.0          bmw x5       good        6.0  gas  145000.0   
 1  25500      2011.0      ford f-150       good        6.0  gas   88705.0   
 2   5500      2013.0  hyundai sonata   like new        4.0  gas  110000.0   
 3   1500      2003.0      ford f-150       fair        8.0  gas  161397.0   
 4  14900      2017.0    chrysler 200  excellent        4.0  gas   80903.0   
 
   transmission    type paint_color  is_4wd date_posted  days_listed  
 0    automatic     SUV         NaN     1.0  2018-06-23           19  
 1    automatic  pickup       white     1.0  2018-10-19           50  
 2    automatic   sedan         red     NaN  2019-02-07           79  
 3    automatic  pickup         NaN     NaN  2019-03-22            9  
 4    automatic   sedan       black     NaN  2019-04-02           28  )

In [3]:
duplicate_count = df.duplicated().sum()
print(f"Total duplicate rows: {duplicate_count}")

Total duplicate rows: 0


There are no duplicate rows

In [4]:
df["odometer_k"] = df["odometer"] / 1000
df["price_k"] = df["price"] / 1000

df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,odometer_k,price_k
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19,145.000,9.4
1,25500,2011.0,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50,88.705,25.5
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79,110.000,5.5
3,1500,2003.0,ford f-150,fair,8.0,gas,161397.0,automatic,pickup,NaN,NaN,2019-03-22,9,161.397,1.5
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28,80.903,14.9


In [5]:
pd.options.display.float_format = '{:,.2f}'.format  #converting the data by decimals makes it easier to read
df. describe()

,price,model_year,cylinders,odometer,is_4wd,days_listed,odometer_k,price_k
count,"51,525.00","51,525.00","51,525.00","51,524.00","25,572.00","51,525.00","51,524.00","51,525.00"
mean,"12,132.46","2,009.79",6.12,"115,199.71",1.00,39.55,115.20,12.13
std,"10,040.80",6.10,1.66,"62,083.20",0.00,28.20,62.08,10.04
min,1.00,"1,908.00",3.00,0.00,1.00,0.00,0.00,0.00
25%,"5,000.00","2,007.00",4.00,"73,500.00",1.00,19.00,73.50,5.00
50%,"9,000.00","2,011.00",6.00,"114,074.00",1.00,33.00,114.07,9.00
75%,"16,839.00","2,014.00",8.00,"152,862.00",1.00,53.00,152.86,16.84
max,"375,000.00","2,019.00",12.00,"990,000.00",1.00,271.00,990.00,375.00


In [6]:
df.isnull().sum()

price               0
model_year          0
model               0
condition           0
cylinders           0
fuel                0
odometer            1
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
odometer_k          1
price_k             0
dtype: int64

In [7]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

#removing outliers from model_year and price
df_cleaned = remove_outliers(df, 'model_year')
df_cleaned = remove_outliers(df_cleaned, 'price')

df_cleaned.info(), df_cleaned.head()

<class 'pandas.core.frame.DataFrame'>
Index: 48718 entries, 0 to 51524
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         48718 non-null  int64  
 1   model_year    48718 non-null  float64
 2   model         48718 non-null  object 
 3   condition     48718 non-null  object 
 4   cylinders     48718 non-null  float64
 5   fuel          48718 non-null  object 
 6   odometer      48718 non-null  float64
 7   transmission  48718 non-null  object 
 8   type          48718 non-null  object 
 9   paint_color   39955 non-null  object 
 10  is_4wd        23705 non-null  float64
 11  date_posted   48718 non-null  object 
 12  days_listed   48718 non-null  int64  
 13  odometer_k    48718 non-null  float64
 14  price_k       48718 non-null  float64
dtypes: float64(6), int64(2), object(7)
memory usage: 5.9+ MB


(None,
    price  model_year           model  condition  cylinders fuel   odometer  \
 0   9400    2,011.00          bmw x5       good       6.00  gas 145,000.00   
 1  25500    2,011.00      ford f-150       good       6.00  gas  88,705.00   
 2   5500    2,013.00  hyundai sonata   like new       4.00  gas 110,000.00   
 3   1500    2,003.00      ford f-150       fair       8.00  gas 161,397.00   
 4  14900    2,017.00    chrysler 200  excellent       4.00  gas  80,903.00   
 
   transmission    type paint_color  is_4wd date_posted  days_listed  \
 0    automatic     SUV         NaN    1.00  2018-06-23           19   
 1    automatic  pickup       white    1.00  2018-10-19           50   
 2    automatic   sedan         red     NaN  2019-02-07           79   
 3    automatic  pickup         NaN     NaN  2019-03-22            9   
 4    automatic   sedan       black     NaN  2019-04-02           28   
 
    odometer_k  price_k  
 0      145.00     9.40  
 1       88.70    25.50  
 2   

In [8]:
fig = px.histogram(df, x="price", nbins=30, color_discrete_sequence=["orange"], title="Distribution of Car Prices")
fig.show()

In [9]:
fig = px.scatter(df, x="odometer", y="price", color_discrete_sequence=["green"], title="Mileage vs Price")
fig.show()

In [10]:
fig = px.bar(df, x="model", color_discrete_sequence=["navy"], title="Number of Cars by Brand")
fig.show()